In [18]:
from treen import load_example
import enflow as ef
import gymnasium as gym

import numpy as np

In [9]:
df = load_example('gefcom2014-wind')

In [10]:
windfarms = []
for site in df.columns.levels[0]:
    windfarms.append(ef.WindFarm(capacity=1, name=site))

portfolio = ef.Portfolio(assets=windfarms)

## Create Environment

In [16]:
dataset = ef.Dataset(name="gefcom2024",
                     description="Data provided by the organisers of HEFTCom2024. Participants are free to use additional external data.",
                     energy_system=portfolio,
                     data={"data_gefcom2014_wind": df})

In [27]:
from typing import SupportsFloat, Sequence, Any
import numpy as np
from numpy.typing import NDArray
from gymnasium.spaces import Box


class DynamicBox(Box):
    """A Box space that supports dynamic shapes in one or more dimensions."""
    
    def __init__(
        self,
        low: SupportsFloat | NDArray[Any],
        high: SupportsFloat | NDArray[Any],
        shape: Sequence[int | None] | None = None,
        dtype: type[np.floating[Any]] | type[np.integer[Any]] = np.float32,
        seed: int | np.random.Generator | None = None,
    ):
        """Initialize the DynamicBox, allowing for None in shape dimensions."""
        
        # Track which dimensions are dynamic (None in the shape)
        self.dynamic_dims = [dim is None for dim in shape] if shape is not None else []
        
        # Create a static shape for initialization, replacing dynamic dimensions with 1
        static_shape = tuple(dim if dim is not None else 1 for dim in shape) if shape is not None else None
        
        # Call the parent Box constructor
        super().__init__(low, high, static_shape, dtype, seed)
        
        # Store the original shape including dynamic dimensions
        self.original_shape = shape

    def _adjust_shape(self, dynamic_shape: Sequence[int] = None):
        """Adjusts the shape by replacing dynamic dimensions with the provided dynamic shape."""
        if dynamic_shape is not None:
            if len(dynamic_shape) != len(self.original_shape):
                raise ValueError(f"Dynamic shape must have the same length as the original shape: {self.original_shape}")
            
            # Replace dynamic dimensions
            adjusted_shape = tuple(
                dynamic_shape[i] if self.dynamic_dims[i] else self.original_shape[i]
                for i in range(len(self.original_shape))
            )
            self._shape = adjusted_shape  # Temporarily set the shape for methods
        else:
            # Revert to the static shape (replace dynamic dimensions with 1)
            self._shape = tuple(1 if self.dynamic_dims[i] else self.original_shape[i] for i in range(len(self.original_shape)))

    def sample(self, dynamic_shape: Sequence[int] = None) -> NDArray[Any]:
        """Generate a random sample from the space, handling dynamic shapes."""
        
        # Adjust shape based on dynamic dimensions
        self._adjust_shape(dynamic_shape)
        
        # Generate sample from adjusted shape
        return super().sample()

    def contains(self, x: Any, dynamic_shape: Sequence[int] = None) -> bool:
        """Check whether a given value is contained within the space, handling dynamic shapes."""
        
        # Adjust shape based on dynamic dimensions
        self._adjust_shape(dynamic_shape)
        
        # Validate the input based on adjusted shape
        return super().contains(x)

    def __repr__(self) -> str:
        """String representation showing dynamic dimensions."""
        dynamic_repr = [
            "None" if is_dynamic else str(dim)
            for is_dynamic, dim in zip(self.dynamic_dims, self.original_shape)
        ]
        return f"DynamicBox({self.low_repr}, {self.high_repr}, {tuple(dynamic_repr)}, {self.dtype})"


In [36]:
state_space = gym.spaces.Dict(
    {
        "wind_power": DynamicBox(low=0, high=1, shape=(None,10))
    }
)
exogeneous_space = gym.spaces.Dict(
    {
        "U10": DynamicBox(low=-np.inf, high=np.inf, shape=(None,1)),
        "V10": DynamicBox(low=-np.inf, high=np.inf, shape=(None,1)),
        "U100": DynamicBox(low=-np.inf, high=np.inf, shape=(None,1)),
        "V100": DynamicBox(low=-np.inf, high=np.inf, shape=(None,1))
    }
)
action_space = gym.spaces.Dict(
    {
        "power_quantile_forecast": DynamicBox(low=0, high=1, shape=(None,99)),
    }
)

In [37]:
class GEFCom2014Wind(gym.Env):
    def __init__(self, dataset: ef.Dataset):
        self.state_space = state_space
        self.exogeneous_space = exogeneous_space
        self.action_space = action_space

        self.train = [[["2012-01-01 01:00:00", "2012-10-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2012-11-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2012-12-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-01-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-02-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-03-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-04-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-05-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-06-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-07-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-08-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-09-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-10-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-11-01 00:00:00"]],
               [["2012-01-01 01:00:00", "2013-12-01 00:00:00"]]],
        self.test = [[["2012-10-01 01:00:00", "2012-11-01 00:00:00"]],
               [["2012-11-01 01:00:00", "2012-12-01 00:00:00"]],
               [["2012-12-01 01:00:00", "2013-01-01 00:00:00"]],
               [["2013-01-01 01:00:00", "2013-02-01 00:00:00"]],
               [["2013-02-01 01:00:00", "2013-03-01 00:00:00"]],
               [["2013-03-01 01:00:00", "2013-04-01 00:00:00"]],
               [["2013-04-01 01:00:00", "2013-05-01 00:00:00"]],
               [["2013-05-01 01:00:00", "2013-06-01 00:00:00"]],
               [["2013-06-01 01:00:00", "2013-07-01 00:00:00"]],
               [["2013-07-01 01:00:00", "2013-08-01 00:00:00"]],
               [["2013-08-01 01:00:00", "2013-09-01 00:00:00"]],
               [["2013-09-01 01:00:00", "2013-10-01 00:00:00"]],
               [["2013-10-01 01:00:00", "2013-11-01 00:00:00"]],
               [["2013-11-01 01:00:00", "2013-12-01 00:00:00"]],
               [["2013-12-01 01:00:00", "2014-01-01 00:00:00"]]]

In [40]:
env = GEFCom2014Wind(dataset) 

In [41]:
env.train

([[['2012-01-01 01:00:00', '2012-10-01 00:00:00']],
  [['2012-01-01 01:00:00', '2012-11-01 00:00:00']],
  [['2012-01-01 01:00:00', '2012-12-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-01-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-02-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-03-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-04-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-05-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-06-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-07-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-08-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-09-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-10-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-11-01 00:00:00']],
  [['2012-01-01 01:00:00', '2013-12-01 00:00:00']]],)